#### Monthly sales report
The sales report is generated at the end of each month for the current month. 
In this notebook I create a simple sales report and save results as excel and html files.
I have to parameterize this notebook so that papermill allows me to execute the same jupyter notebook with different variables (different sales data for each month).

In [1]:
# Import libraries
import pandas as pd
import random
import datetime
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.offline as pyo
import calendar

In [2]:
filename='/home/kinga/python/Papermill-report/Data_August.xlsx'

In [3]:
# Parameters
# This template report uses sales data for August. Thanks to papermill I will execute this notebook 
# every month with different data for each subsequent month (for example at the end of September
# with data for September etc)
filename = "Data_August.xlsx"


In [4]:
df=pd.read_excel(filename)

#### Report  August

In [5]:
# report is generated for the current month, so I have to find out what the cureent month is
current_date=datetime.date.today()
current_month=current_date.month
month_name=calendar.month_name[current_month]
current_year=current_date.year
print ('current month is:', month_name, ', current year is:', current_year)


current month is: August , current year is: 2020


In [6]:
# selecting data for the current month 
df_current=df[pd.to_datetime(df['Date']).dt.month == current_month]
df_current.head()

,Unnamed: 0,Name,Date,Quantity_prod_A,Quantity_prod_B,Quantity_prod_C,Total
1,1,Filip,2020-08-16,9,1,4,14
5,5,Karolina,2020-08-28,3,8,7,18
12,12,Filip,2020-08-11,5,4,4,13
20,20,Adam,2020-08-28,2,1,8,11
25,25,Karolina,2020-08-10,4,6,9,19


In [7]:
# quick statistical analysis
statistic=df_current.describe().iloc[:,1:]
# results are saved in the excel file. There will be a file with a proper month name 
statistic.to_excel('/home/kinga/python/Papermill-report/monthly_reports/'+month_name+'-statistic.xlsx')
statistic

,Quantity_prod_A,Quantity_prod_B,Quantity_prod_C,Total
count,332.000000,332.000000,332.000000,332.00000
mean,5.548193,5.737952,5.183735,16.46988
std,2.882507,2.795228,2.883326,5.15989
min,1.000000,1.000000,1.000000,4.00000
25%,3.000000,3.750000,3.000000,13.00000
50%,6.000000,6.000000,5.000000,17.00000
75%,8.000000,8.000000,8.000000,20.00000
max,10.000000,10.000000,10.000000,29.00000


In [8]:
# sales by Representative in the current month 
sales_by_rep=pd.pivot_table(df_current, index='Name', values=["Quantity_prod_A", "Quantity_prod_B", "Quantity_prod_C",
                            "Total"], aggfunc=np.sum, margins=True)
sales_by_rep=pd.DataFrame(sales_by_rep.to_records())
# results are saved in the excel file. There will be a file with a proper month name 
sales_by_rep.to_excel('/home/kinga/python/Papermill-report/monthly_reports/'+month_name+'-sales-by-rep.xlsx')
sales_by_rep

,Name,Quantity_prod_A,Quantity_prod_B,Quantity_prod_C,Total
0,Adam,361,349,337,1047
1,Filip,475,496,458,1429
2,Karolina,272,313,251,836
3,Katarzyna,391,405,357,1153
4,Robert,343,342,318,1003
5,All,1842,1905,1721,5468


In [9]:
# Sales by Representative on the bar chart. Results are save as html file
data=[]

products=sales_by_rep.columns[1:4] 

for prod in products:
    trace=go.Bar(x=sales_by_rep['Name'].iloc[:-1], y=sales_by_rep[prod],text=sales_by_rep[prod],
                 textposition='auto',orientation='v', name=prod)
    data.append(trace)
    
layout=go.Layout(title='Sales by Representative in '+month_name, barmode='group', hovermode='closest',
                 bargroupgap=0.1,xaxis={'title': 'Representative' }, yaxis={'title': 'Sales' })



fig=go.Figure(data=data, layout=layout)
#There will be a file with a proper month name
pyo.plot(fig, filename='Sales_by_rep_'+month_name+'.html')

'Sales_by_rep_August.html'